# Seasonal variation of CO2 over the Southern Ocean from aircraft observations

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import xarray as xr

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import datasets
import emergent_constraint as ec
import figure_panels
import obs_aircraft
import obs_surface
import util

## Compute trends at SPO 

Identify the models we're using the analysis.

In [ ]:
campaign_info = obs_aircraft.get_campaign_info(verbose=False, lump_orcas=True)

Load the observations and "simulated" observations.

In [ ]:
da_srf = obs_surface.open_surface_co2_data('obs', 'CO2')
da_srf = da_srf.sel(record='SPO_NOAA_insitu_CO2')
da_srf

In [ ]:
spo_fit = util.apply_stl_decomp(da_srf)
spo_fit

Visualize the fit

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(6, 8), squeeze=False,)

spo_fit.observed.plot(label='Data', ax=axs[0, 0])
spo_fit.trend.plot(label='STL trend', ax=axs[0, 0])
axs[0, 0].set_ylabel('CO$_2$ [ppm]')
axs[0, 0].set_xlabel('')
axs[0, 0].set_xticklabels([])
axs[0, 0].legend()
axs[0, 0].set_title('SPO CO$_2$ record', loc='left')

spo_fit.seasonal.plot(ax=axs[1, 0])
axs[1, 0].axhline(0, linewidth=0.5, color='k')
plt.title('Seasonal component of STL fit')
axs[1, 0].set_ylabel('Seasonal CO$_2$ [ppm]')
axs[1, 0].set_xlabel('')
axs[1, 0].set_xticklabels([])
axs[1, 0].set_title('STL-diagnosed seasonal cycle', loc='left')

spo_fit.resid.plot(ax=axs[2, 0])
axs[2, 0].axhline(0, linewidth=0.5, color='k')
axs[2, 0].set_ylabel('Residual CO$_2$ [ppm]')
axs[2, 0].set_xlabel('')
axs[2, 0].set_title('STL-residuals', loc='left')
ylm = np.array(axs[2, 0].get_ylim())
axs[2, 0].text(
    axs[2, 0].get_xlim()[0]+200, ylm[1]-np.diff(ylm)*0.15, 
    f'r$^2$ = {util.r2(spo_fit):0.4f}',
    fontweight='bold',
    ha='left',
);

In [ ]:
dsets_prof = datasets.aircraft_profiles('obs')[['co2_med']]
dsets_prof

In [ ]:
spo_trend = xr.DataArray(
    np.interp(dsets_prof.time, spo_fit.time, spo_fit.trend),
    dims=('profile'), 
)
dco2 = dsets_prof.co2_med - spo_trend
dco2

In [ ]:
xhat = np.arange(-5, 365+5, 1)    
yhat = xr.DataArray(np.ones((len(dco2.theta), len(xhat)))*np.nan, 
                    dims=('theta', 'doy'),
                    coords=dict(
                        theta=dco2.theta,
                        doy=xhat,
                    ),
                   )

doy = dco2.doy.values
for k in range(len(dco2.theta)):
    x, y = util.antyear_daily(doy, dco2.isel(theta=k).values)
    missing = np.isnan(x) | np.isnan(y)
    if np.sum(~missing) < 5:
        continue
    p, pcov = ec.curve_fit(
        ec.harmonic, 
        xdata=x[~missing]/365.25, 
        ydata=y[~missing], 
    )

    yhat.data[k, :] = ec.harmonic(xhat/365.25, *p)

    
seasonal_amplitude = (yhat.max('doy') - yhat.min('doy'))
yhat.plot()

In [ ]:
fig = plt.figure(figsize=(8, 4)) #dpi=300)

c_spec = figure_panels.marker_spec_campaigns(lump_orcas=True)
txt_box_props = dict(boxstyle='square,pad=0', facecolor='none', edgecolor='none')

# set up plot grid
gs = gridspec.GridSpec(
    nrows=1, ncols=4, 
    width_ratios=[0.65, 0.015, 0.075, 0.375],
    left=0., right=1.,
    bottom=0.05, top=0.95,
    hspace=0.25, wspace=0.05,
)
axs = np.empty((1, 2)).astype(object)
axs[0, 0] = plt.subplot(gs[0, 0])
cax = plt.subplot(gs[0, 1])
axs[0, 1] = plt.subplot(gs[0, 3])

ax = axs[0, 0] 
pc = ax.contourf(
              yhat.doy,
              yhat.theta.sel(theta=slice(None, 320)), 
              yhat.sel(theta=slice(None, 320)),
              levels=figure_panels.levels,
              norm=figure_panels.divnorm,
              cmap=figure_panels.cmap,
             )

for c, info in campaign_info.items():
    tb = util.day_of_year(info['time_bound'])
    x, _ = util.antyear_daily(tb, np.ones(2))
    ax.axvspan(
        x[0], x[1], 
        ymin=0.84 if c == 'ORCAS' else 0.86, 
        ymax=0.98,
        color=c_spec[c]['color'], 
        alpha=1 if c in ['HIPPO-1', 'ATOM-2'] else 0.75, 
    )
    xtxt = x.mean() + np.diff(x)*0.18
    if c == 'ORCAS':
        xtxt += 10
    ax.text(
        xtxt, 320., c,
        rotation=90, 
        ha='center', 
        verticalalignment='top', 
        color='k', 
        fontsize=7,
        bbox=txt_box_props,
    )
    
    
ax.set_ylim((269., 321.))
ax.set_title('Interpolated obs: CO$_2$ minus SPO trend')


cb = plt.colorbar(pc, cax=cax)
cb.ax.set_title('$\Delta$CO$_2$ [ppm]      ', loc='center')

ax.set_ylabel('$\\theta$ [K]')

ax.set_xlim((-10, 375))
ax.set_xticks(figure_panels.bomday)
ax.set_xticklabels([f'        {m}' for m in figure_panels.monlabs_ant]+['']);

ax = axs[0, 1]
ax.plot(
    seasonal_amplitude.sel(theta=slice(None, 320)), 
    yhat.theta.sel(theta=slice(None, 320)), 
    '.-', label='Seasonal amplitude')

ax.set_yticklabels([]);
ax.set_xlabel('Seasonal amplitude [ppm]')
ax.axvline(seasonal_amplitude.sel(theta=300.), lw=0.5, c='dimgray', linestyle='--')
ax.set_ylim((269., 321.))
ax.set_title('Amplitude of seasonal cycle')
util.label_plots(fig, [ax for ax in axs.ravel()], xoff=-0.02)

util.savefig('seasonal-amplitude')